In [1]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
camera = cv2.VideoCapture(0)
time.sleep(0.25)

firstFrame = None
runsim = 0

min_area = 10000

while(True):
    (grabbed, frame) = camera.read()
    frame = cv2.resize(frame, (480,360))
    
    text = "No Detection"
    
    if not grabbed:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    gray = cv2.GaussianBlur(gray, (3,3), 0)
    
    bg = cv2.waitKey(1) & 0xFF
    
    if bg == ord("c"):
        print("OK")
        firstFrame = gray
        runsim = 1
        continue
    
    if runsim == 1:
        frameDelta = cv2.absdiff(firstFrame, gray)
        
        thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]
        
        kernel1 = np.ones((11,11),np.uint8)
        kernel2 = np.ones((9,9),np.uint8)
        
        erosion =cv2.erode(thresh, kernel1, iterations=1)
        
        dilation = cv2.dilate(erosion, kernel2, iterations=2)
        
        _, cnts, _ = cv2.findContours(dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for c in cnts:
            
            if cv2.contourArea(c) < min_area:
                continue
            
            
            (x,y,w,h) = cv2.boundingRect(c)
            
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            
            text = "Detecting"
        
        
        
        cv2.putText(frame,"Status: {}".format(text), (10,20), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
        
        horizontal1 = np.hstack((firstFrame, frameDelta))
        horizontal2 = np.hstack((thresh,erosion, dilation))
        cv2.imshow("FirstFrame & Frame Different", horizontal1)
        cv2.imshow("Thresh ----- Erosion ----- Dilation", horizontal2)
        # cv2.imshow("Thresh", thresh)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
    
camera.release()
cv2.destroyAllWindows()